In [1]:
# The code was removed by Watson Studio for sharing.
# Load dataframe

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200516135437-0001
KERNEL_ID = af72c332-d42c-4184-a6f4-b0743b52ba66


In [2]:
# Since all data came in as string we convert them back
from pyspark.sql.functions import col
df = df.select(
        col('TMC').cast('double'),
        col('Severity').cast('int'),
        col('Start_Time').cast('timestamp'),
        col('Start_Lat').cast('double'),
        col('Start_Lng').cast('double'),
        col('Distance(mi)').cast('double'),
        col('Number').cast('double'),
        col('Street').cast('string'),
        col('Side').cast('string'),
        col('City').cast('string'),
        col('County').cast('string'),
        col('State').cast('string'),
        col('Zipcode').cast('string'),
        col('Country').cast('string'),
        col('Timezone').cast('string'),
        col('Airport_Code').cast('string'),
        col('Weather_Timestamp').cast('timestamp'),
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'),
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),
        col('Precipitation(in)').cast('double'),
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    )

# Functions used to simplify
Some of the following functions are deprecated since they wont work on the cloud (create spark session, save logs and model)

In [3]:
from pyspark.sql.functions import when, count, col, isnan, countDistinct,from_unixtime,from_utc_timestamp, unix_timestamp,split, to_timestamp, hour, month, lit,collect_list, max
import seaborn as sns 
import matplotlib.pyplot as plt 
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.ml.tuning import CrossValidator
from datetime import datetime
from numpy import argmax
from sklearn.metrics import classification_report, confusion_matrix

class logging():
    def __init__(self, models_dir, logs_dir, image_dir, foldername, enabled=False):
        self.timeSignature = str(datetime.now().strftime("%Y%m%d%H%M%S"))
        foldername = foldername + "_" +self.timeSignature
        self.models_dir = models_dir + "/" +foldername +"/" 
        self.logs_dir = logs_dir + "/" +foldername + "/logs.md" 
        self.image_dir = image_dir +"/" + foldername +"/" 
        
        self.enabled = enabled
        
        self.createDirectory(self.logs_dir)
        
    
    def createDirectory(self,folder):
        """
        Creates empty folder if it does not exist

        Parameters
        ----------
        folder: string
            Path to folder
        Return
        ----------
        """
        if self.enabled==False: return None
        import os
        os.makedirs(os.path.dirname(folder))
        
        
    def write2file(self, title, info, logs_dir=None):
        """
        Write data to filelog

        Parameters
        ----------
        file: string
            Path to file
        title: string
            Title for info
        info: string
            Data that shall be stored

        Return
        ----------
        """
        
        if self.enabled==False: return None
        if logs_dir == None: logs_dir = self.logs_dir
            
        f = open(logs_dir, "a")
        f.write(f"\n\n# {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} \n__{title}:__\n{info}")
        f.close()
        
    def saveImage(self, image, name, image_dir=None):
        """
        Save file to disk
        Parameters
        ----------
        image: obj
            Image object
        name: string
            Name of file
        image_dir: string, Optional, Default=None
            Alternative path to file

        Return
        ----------
        """
        if self.enabled==False: return None
        if image_dir == None: image_dir = self.image_dir
        try:
            image.figure.savefig(image_dir +"/" + name)
        except:
            image.savefig(image_dir +"/" + name)
            
    def write2fileModel(self, model, model_name, models_dir=None):
        """
        Write the model to filesystem

        Parameters
        ----------
        model: obj
            Model object

        models_dir:
            Directory for the model

        model_name: string
            Name of the model

        timeSignature: string
            Time when program started

        Return
        ----------
        """
        if self.enabled==False: return None
        if models_dir == None: models_dir = self.models_dir
            
        print(f"Saving into: {models_dir}{model_name}")
        try:
            model.write.format('parquet').mode('overwrite').option("header", "true").save(f"{models_dir}/{model_name}")
            print("Saving through format 1")
        except Exception as e:
            print(e)
            model.write().overwrite().save(f"{models_dir}/{model_name}")
            print("Saving through format 2")
           
    def setLogDir(self, logs_dir):
        self.logs_dir = logs_dir
    
    def setModelsDir(self, models_dir):
        self.models_dir = models_dir

def setup_spark(file):
    """
    Setup spark config and server and load datafile
    
    Parameters
    ----------
    file: string
        File to be loaded
    
    Return
    ----------
    """
    try:
        spark.close()
    except: 
        pass

    try:
        conf = SparkConf().setAppName('Spark Project')
        conf.set('spark.driver.memory', '6g')
        conf.set('spark.executor.memory', '5g')
        conf.set('spark.driver.maxResultSize','2g')
        conf.set('sql.autoBroadcastJoinThreshold','-1')
        #conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        #conf.set("spark.kryoserializer.buffer.max", "256m")
        conf.set('spark.ui.enabled', 'true')
        conf.set('spark.ui.killEnabled', 'false')
        conf.set("spark.memory.offHeap.enabled",'true')
        conf.set("spark.memory.offHeap.size","4g") 
        #conf.set('spark.serializer.objectStreamReset', '100')
        #conf.set('spark.driver.cores', '1')
        #conf.set('spark.executor.id', 'driver')
        #conf.set('spark.executor.cores', '1')
        #conf.set('spark.ui.showConsoleProgress', 'true')
        #conf.set('spark.executor.instances', '1')
        #conf.set('spark.rdd.compress', 'True')
        sc = SparkContext(conf=conf)
        
        spark = SparkSession.builder \
        .appName("Spark Project") \
        .master("local[*]") \
        .getOrCreate()
        spark.sparkContext.setLogLevel("WARN")
        
    except Exception as e:
        print(e)

    df = spark.read.load(file,format="csv", sep=",", inferSchema="true", header="true")
    return df, sc, spark

def setup_variables(df, sc, colLabel, colRem):
    """
    Creates the column items for category and numerical values.
    
    Parameters
    ----------
    df: Dataframe
        Spark Dataframe
    sc: SparkContext object
        SparkContext object
    colLabel: List
        Items that considered Label
    colRem: List
        Items that should be removed before analysis
    logs_dir: string
        Directory for log file
    Return
    ----------
    df: Dataframe
        Spark Dataframe
    colCat: List
        Items that is considered categories
    colNum: List
        Items that is considered numerical values
    """
    # Dropping data that cant help during model
    df = df.drop(*colRem)

    # Convert boolean to string since PCA cant handle boolean which should be a class
    df = df.select(*[col(c[0]).cast("string").alias(c[0]) if c[1] == 'boolean' else col(c[0]).alias(c[0]) for c in df.dtypes])

    #renamedHousing.select([count(when(col(c).isNull(), c)).alias(c) for c in colNum]).show()
    colCat, colNum = createNewClasses(df, sc, colLabel)

    return df, colCat, colNum

def createNewClasses(df, sc, colLabel):
    """
    Divide the data into classes
    
    Parameters
    ----------
    df: Dataframe
        Spark Dataframe
    sc: SparkContext object
        SparkContext object
    colLabel: List
        Items that considered Label
        
    logs_dir: string
        Directory for log file
        
    Return
    ----------
    colCat: List
        Items that is considered categories
    colNum: List
        Items that is considered numerical values
    """
    rdd = sc.parallelize(df.dtypes)
    
    colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()
    colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()
    
    print(f"Label: {colLabel} \nCategories: {colCat}\nNumerical: {colNum}")
    return colCat, colNum 

def printMissingValues(df, logger):
    """
    Print missing values in Dataframe
    
    Parameters
    ----------
    df: Dataframe
        Spark Dataframe
    logs_dir: string
        Directory for log file
        
    Return
    ----------
    """
    # Check data
    df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Missing value in each column
    tmp = df_missingVals_cols.collect()
    logger.write2file("Missing values", str(tmp))
    print("Missing values" + str(tmp))
    
def printCategoricalValues(df, colCat, logger):
    """
    Print the count of unique values from categories
    
    Parameters
    ----------
    df: Dataframe
        Spark Dataframe
    colCat: List
        Items that is considered categories
    logs_dir: string
        Directory for log file
        
    Return
    ----------
    """
    # Checking how many classes that can be used
    tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in colCat] 
    print("Unique column values:", tmp)

    logger.write2file("Categorical values", str(tmp))

def evaluateModel(estimator, 
                  paramGrid, 
                  modelType, 
                  trainSet, 
                  testSet, 
                  evaluator, 
                  k=10,
                  seed=None,
                  logger=None):
    """
    Evaluate model through CrossValidator, save model, and check accuracy
    
    Parameters
    ----------
    estimator: Obj
        Algorithm used for fitting model
    paramGrid: paramGrid
        Params that will be tested and evaluated
    modelType: string
        Name of the model
    trainSet: Dataframe
        Training set
    testSet: Dataframe
        Test set
    timeSignature: string
        Time when program started
    evaluator: Obj, Optional, Default: MulticlassClassificationEvaluator
        Evaluator
    k: int
        Number of folds
    seed:
        Seed
    logs_dir: string
        Directory for log file
    models_dir:
        Directory for the model
    Return
    ----------
    model: Obj
        Model that is fitted
    predictions: obj
        Predictions that is transformed based on the model and testSet
    """
    cv =  CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=k, seed=seed) 
    
    print("Fitting model..")
    logger.write2file("Fitting "+str(modelType)+" model..","")
    
    model = cv.fit(trainSet)
    
    print("Predicting on testSet..")
    logger.write2file("Predicting "+str(modelType)+" on testSet..","")
    
    predictions = model.transform(testSet)
    samplePredict = predictions.select("prediction", "label", "features")
    
    print("Evaluating predictions..")
    logger.write2file("Evaluating "+ str(modelType) +" predictions..","")
    
    accuracy = evaluator.evaluate(predictions)
    params = model.getEstimatorParamMaps()[argmax(model.avgMetrics)]
    
    logger.write2file("Evaluating "+ str(modelType),"Display 5 datapoints:\n" + str(samplePredict.take(5)) + "\nAccuracy: " + str(accuracy)+"\nParameters:\n" + str(params))
    logger.write2fileModel(model, str(modelType))
    samplePredict.show(5)
    print("Accuracy: ",accuracy, "\nParameters\n",params)

    y_true = predictions.select(['label']).collect()
    y_pred = predictions.select(['prediction']).collect()
    try:
        print(f"Classification report:\n{classification_report(y_true, y_pred)}")
        print(f"Confusion matrix:\n{confusion_matrix(y_true, y_pred)}")
    except:
        print('The cloud does not support Classification report or confusion matrix')
   
    return model, predictions

In [4]:
from datetime import datetime
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.ml.linalg import SparseVector, VectorUDT
from pyspark.ml import Pipeline
from pyspark.ml.feature import ChiSqSelector,PCA

from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator

# Setup environment

In [5]:
model_indexing_dir = "log_index.md"
images_dir = "analysis"
models_dir = "models"
logs_dir = "models" 
filter_state = 'CA' # ALL if all states
model_Note = ''
log_mode=False # Logs data to files, Dont use on massive dataset (debug_mode is recommended while on)
debug_mode=False # Reduce number of rows collected for faster processing
enable_plots = True # Dont use unless debug_mode is on, or datasize is small
logger = logging(models_dir, logs_dir, "", filter_state, enabled=log_mode)

file = "data/US_Accidents_Dec19.csv"


df, sc,spark = setup_spark(file)

Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=spark://jkg-deployment-af72c332-d42c-4184-a6f4-b0743b52ba66-7f657fn9cs9:7077) created by getOrCreate at /opt/ibm/kernelScript/python/shell.py:65 


UnboundLocalError: local variable 'spark' referenced before assignment

# Define variables

In [ ]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem)  

logger.write2file("Number of rows", str(df.count()))
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

In [ ]:
sc

In [ ]:
tmp = sc._conf.getAll()
logger.write2file("New Spark session", str(tmp))
print("Config:",tmp)

# 1. Analysis 

In [ ]:
print(f"Categories:\n \
Labels: {len(colLabel)}\n \
Classes: {len(colCat)}\n \
Removed: {len(colRem)}\n \
Numerical: {len(colNum)}") 

info = f"Rows: {df.count()}\nColumns {len(df.columns)}"
print(info)
df.printSchema()
df.take(1)

logger.write2file("Data analysis",info +"\n"+ str(df._jdf.schema().treeString()) + str(df.take(1)))

## Statistical summary

In [ ]:
df.describe(colNum).show()

## Get missing data

In [ ]:
printMissingValues(df, logger)

## Get categorical values

In [ ]:
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat]] 
df.unpersist()
print("Unique column values:", tmp)

logger.write2file("Unique column values", str(tmp))

## Correlation

In [ ]:
print([*colNum, *colLabel])

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

assembler = VectorAssembler(
    inputCols=colNum,
    outputCol="features",
    handleInvalid = "skip")

df_attributes = assembler.transform(df)
df_attributes.select("features").show(1,False)



r1 = Correlation.corr(df_attributes, "features").head()

print("correlation matrix:\n" + str(r1[0]))

logger.write2file("Correlation matrix", str(r1[0]))

In [ ]:
if enable_plots == True:
    sns.set(style="white")
    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    b = sns.heatmap(r1[0].toArray().tolist(), annot=True, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5},xticklabels=colNum,yticklabels=colNum,ax=ax )
    ax.set_title("Correlation between numerical features")

    logger.saveImage(b,"feature_corr_matrix")

## Severity

In [ ]:
severity_freq = df.groupBy('Severity').count().orderBy('count',ascending=False)
severity_freq.take(10)

In [ ]:
if enable_plots == True:
    pd_severity = severity_freq.toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(pd_severity['Severity'],pd_severity['count'], color='blue')
    b.axes.set_title("Severity distribution",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("Severity",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist")

In [ ]:
if enable_plots == True:
    df_sev = df.groupby('Severity').count().toPandas()
    fig = df_sev.plot.pie(y='count', labels=df_sev['Severity'], figsize=(10, 10), autopct='%1.0f%%',title="Pie plot - Severity distribution",fontsize=15)
    logger.saveImage(fig,"pie_severity_dist")
    

## Causes

### Frequency of severity and state

In [ ]:
state_freq = df.groupBy('State').count().orderBy('count',ascending=False)
state_freq.take(10)

In [ ]:
if enable_plots == True:
    pd_states = state_freq.toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(pd_states['State'],pd_states['count'], color='blue')
    b.axes.set_title("Severity distribution for each state",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("State",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist_state")

In [ ]:
state_severity_freq = df.groupBy('State','Severity').count().orderBy('count',ascending=False)
state_severity_freq.take(10)

In [ ]:
if enable_plots == True:
    pd_state_severity = state_severity_freq.toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(x="State", y="count", hue="Severity", data=pd_state_severity)
    #b = sns.barplot(pd_state_severity['State', 'Severity'],pd_state_severity['count'])
    b.axes.set_title("Severity distribution for each state",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("State",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist_class_state")

### Weather conditions

In [ ]:
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
weather_freq.take(10)

In [ ]:

n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

rdd_filtered = weather_freq.rdd.filter(lambda x: x['count'] > n)
print(rdd_filtered.collect())

if enable_plots == True:
    fig, ax=plt.subplots(figsize=(16,25))
    pd_weather = rdd_filtered.toDF().toPandas()

    b = sns.barplot(pd_weather['count'][:],pd_weather['Weather_Condition'][:], color="blue")

    b.axes.set_title("Weather Condition for accidents above 1% of total set",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("Weather_Condition",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"weather_cond_dist")
    logger.write2file("Weather condition distribution", str(pd_weather))

### Time when accidents occured

In [ ]:
df_time = df.selectExpr("hour(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False)
time_freq.take(10)


In [ ]:
if enable_plots == True:
    pd_time = time_freq.toPandas()
 
    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))

    b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
    b.axes.set_title("Daytime for accidents",fontsize=20)
    b.set_xlabel("Time (Hours)",fontsize=15)
    b.set_ylabel("Number of Accidents",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"accident_hours")

### Development of accidents on Month basis

In [ ]:
df_time = df.selectExpr("month(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False)
time_freq.take(10)

In [ ]:
if enable_plots == True:
    pd_time = time_freq.toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))

    b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
    b.axes.set_title("Montly distribution for accidents",fontsize=20)
    b.set_xlabel("Time (Month)",fontsize=15)
    b.set_ylabel("Number of Accidents",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"accudents_months")

### Map distribution

In [ ]:
if enable_plots == True:
    b = sns.jointplot(x=df.select(collect_list('Start_Lat')).first()[0],y=df.select(collect_list('Start_Lng')).first()[0],height=10)

    b.set_axis_labels('Start_Lat','Start_Lng')
    b.fig.suptitle("Map distribution of accidents")
    logger.saveImage(b,"accident_map_dist")

In [ ]:
logger.write2file("New Analysis created!", \
               "Model finished: Yes" + \
               "\nFolder name: " + logger.timeSignature + \
               "\nState: "+ "Not defined" + \
               "\nLogs directory: " + logger.logs_dir +  \
               "\nFile: " + file + \
               "\nNote: " + model_Note, \
               logs_dir=model_indexing_dir \
              )

# 2. Preprocessing

## Analyse single state
#This could be CA since it stands for a little less than 50% of the total set

In [ ]:
if filter_state != 'ALL':
    df = df.filter(df.State == filter_state) # Lowers the dataset quite a lot
    logger.write2file("Specified state",str(filter_state))
else:
    logger.write2file("No state specified","")

## Modify time
#Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [ ]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)
logger.write2file("Dataset after modifying UTC timestamp", str(df.take(1)))

## Cluster quantiles to reduce the feature vector for position
#Since feature crossing wont work due to the feature vector is to big (regarding to the given resources) we hope that Clustering might reduce them and keep as much data as possible

In [ ]:
number_of_clusters= 400
pos_assembler = VectorAssembler(inputCols=['Start_Lat', 'Start_Lng'], outputCol="position_features")
preprocessed_cord = pos_assembler.transform(df)

trainSet, testSet = preprocessed_cord.randomSplit([0.9 ,0.1], 1)
kmeans = KMeans(k=number_of_clusters, featuresCol='position_features', predictionCol='position')
model = kmeans.fit(trainSet)
predictions = model.transform(testSet)

evaluator = ClusteringEvaluator(featuresCol='position_features', predictionCol='position')

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Transform the hole dataset
df = model.transform(preprocessed_cord)
df.select('position').show(5)

In [ ]:
df.show(1)
logger.write2file("Dataset after adding quantiles",str(df.take(1)))

## Cast data 
#Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too #many NaN values or were converted into another form.

In [ ]:
# Could be handles as separated lists and then sent to a UDF for processing but since low number of features and easier to visualize this select is kept.
df = df.select(
        col('Severity').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        col('position').cast('string'),
        col('Distance(mi)').cast('double'),
        col('Side').cast('string'),   
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), 
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'), 
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

colLabel = ["Severity"]

colCat, colNum = createNewClasses(df, sc, colLabel)
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

## Get categorical values

In [ ]:
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat]] 
df.unpersist()
print("Unique column values:", tmp)

logger.write2file("Unique column values", str(tmp))

## Clean data

### Recheck the missing values
#Check so that the output contains 0 missing values

In [ ]:
df.persist(StorageLevel.MEMORY_AND_DISK)
printMissingValues(df,logger)
df.unpersist()

### Remove data with occurance less than 1%
#Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [ ]:
df.persist()
original_rows = df.count()
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
df_filtered = weather_freq.filter(weather_freq['count'] > n)
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
removed_rows = original_rows - df.count()

df.unpersist()
print("Rows removed:",removed_rows)

logger.write2file("Weather condition", str(df_filtered.take(df_filtered.count())) + "\nRows removed: " + str(removed_rows))

# Remove if distict values in category equals 1

In [ ]:
df.persist()
tmp = [c for c in [*colCat] if df.select(countDistinct(c)).collect()[0][0] <= 1] 
df.unpersist()

[colCat.remove(c) for c in tmp]
print("Dropping columns with 1 class:", tmp)
logger.write2file("Dropping columns with 1 class", str(tmp))

## Prepare Pipeline

In [ ]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='skip') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers])

## Preprocessing - Pipeline

In [ ]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])

preprocessed_df = pipeline.fit(df).transform(df)

In [ ]:
preprocessed_df.select("scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

## Combine features

In [ ]:
va2 = VectorAssembler(inputCols=["scaledFeatures", *[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)
df.persist()
df = df.withColumn('label', col("Severity"))
df = df.withColumn('features', df.final_features).select("features","label")
df.unpersist()


logger.write2fileModel(df, "df_features")

logger.write2file("Feature set size", str(df.count()) + "\n\n__Feature vector and label:__\n" + str(df.take(1)))
logger.write2file("Number of rows", str(df.count()))
print("Feature set size: ",df.count())

## Count categorical values

In [ ]:
# Checking how many classes that can be used
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat, "position"]] 
df.unpersist()
print("Unique column values:", tmp)

#logger.write2file("Unique column values", str(tmp))

# 3. Feature importance

## PCA
#After editing Quantilization PCA started to crash by giving error: spark.driver.maxResultSize
#In order to understand how much the variance affect the dataset we check with PCA. Try to get above 90%

In [ ]:
length_of_feature_vec = 250
pca = PCA(k=length_of_feature_vec, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df)
pca_df = pca_model.transform(df)


print("PCA - Feature Variance:","Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+str(k)+"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2file("PCA - Feature variance", "Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+ str(k) +"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
logger.write2fileModel(pca_df, "pca_df")

## ChiSqSelector
#Check top 100 which of the values in the feature vector

In [ ]:
selector = ChiSqSelector(numTopFeatures=length_of_feature_vec, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

chi_model = selector.fit(df)
chi_df = chi_model.transform(df)

logger.write2file("Top selected features according to ChiSqSelector", str(chi_model.selectedFeatures)+ "\nNumber of features: " + str(k) + "\nExample data:\n"+str(chi_df.take(5)))
logger.write2fileModel(chi_df, "chi_df")

print("Top selected features according to ChiSqSelector:", chi_model.selectedFeatures)
print("Transformed selected features:",chi_df.head().selectedFeatures)

# 4. Machine learning

In [ ]:
trainSet, testSet = chi_df.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
trainSet.show(2,False)

## Logistic regression

In [ ]:
lr = LogisticRegression(labelCol="label", featuresCol="selectedFeatures")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1,0.01]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
try: 
    model, _ = evaluateModel(lr, paramGrid, "LR_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)
except Exception as e:
    logger.write2file("Error:", str(e))
    print(e)

## Decision tree

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="selectedFeatures") 

paramGrid = ParamGridBuilder().build()

_, _ = evaluateModel(dt, paramGrid, "DT_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

## Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="selectedFeatures") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[15]) \
                .build()

_, _ = evaluateModel(rf, paramGrid, "RF_Model", trainSet, testSet, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logger=logger)

In [ ]:
logger.write2file("Program finished!", "")

In [ ]:
logger.write2file("New model folder created", \
               "Model finished: Yes" + \
               "\nFolder name: " + logger.timeSignature + \
               "\nState: "+ filter_state + \
               "\nLogs directory: " + logger.logs_dir +  \
               "\nFile: " + file + \
               "\nNote: " + model_Note, \
               logs_dir=model_indexing_dir \
              )
